In [1]:
import pandas as pd
import os

data_path = '/nobackup/users/hmbaghda/metastatic_potential/'

### Table WW

- Sheet 1: Consensus model hyperparameters selected (from results reported in Table WW)
- Sheets 2-4: In each of 10-folds, the best performing hyper-parameters identified in Optuna from inner 5-fold CV and the performance metrics for that model in that fold.
- Sheet 5: Hyperparameters used in consensus models

#### Sheet 1:

In [55]:
hps = pd.DataFrame(columns = ['hyperparameter', 'relevant models', 'values explored'])

hps.loc[0,:] = ['Number of Selected Features', 
                'all', 
                '250, 500, 1000, 5000, all features']

hps.loc[1, :] = ['C', 'SVR', 'between 1e-4 and 1e-2, log = True']
hps.loc[2, :] = ['epsilon', 'SVR', 'between 1e-3 and 10, log = True']
hps.loc[3, :] = ['gamma', 'SVR (rbf and poly kernels only)', "'scale', 'auto'"]
hps.loc[4, :] = ['degree', 'SVR (poly kernel only)', "2,3,4,5"]
hps.loc[5, :] = ['coef0', 'SVR (poly kernel only)', 'between 0 and 2 with a step size of 0.1']

hps.loc[6, :] = ['n_components', 'PLSR', 'between 2 and 100 with a step size of 1']
hps.loc[7, :] = ['alpha', 'Ridge, Lasso, ElasticNet', 'between 1e-3 and 1e2, log = True']
hps.loc[8, :] = ['l1_ratio', 'ElasticNet', 'between 0.3 and 0.7 with a step size of 0.1']

hps.loc[9, :] = ['n_estimators', 'Random Forest', 'between 300 and 1600 with a step size of 400']
hps.loc[10, :] = ['max_features', 'Random Forest', '"sqrt", "log2", 0.5, 0.75, 1']
hps.loc[11, :] = ['max_samples', 'Random Forest', '0.25, 0.5, 0.75, None']
hps.loc[12, :] = ['max_depth', 'Random Forest', 'None, 10, 25, 50, 100, 200']

hps.loc[13, :] = ['n_neighbors', 'KNN', 'between 15 and 25 with a step size of 1']
hps.loc[14, :] = ['weights', 'KNN', "'uniform', 'distance'"]
hps.loc[15, :] = ['metric', 'KNN', "'minkowski', 'l1', 'l2', 'cosine'"]

#### Sheet 5: 

In [56]:
rna_hp = pd.read_csv(os.path.join(data_path, 'interim', 
                                       'pipeline_model_selection_transcriptomics_individual.csv'), 
                    index_col = 0)

protein_hp = pd.read_csv(os.path.join(data_path, 'interim', 
                                       'pipeline_model_selection_proteomics_individual.csv'), 
                    index_col = 0)

joint_hp = pd.read_csv(os.path.join(data_path, 'interim', 
                                       'pipeline_model_selection_joint_individual.csv'), 
                    index_col = 0)


In [57]:
hp_all = {'transcriptomics': rna_hp, 
         'proteomics': protein_hp, 
         'joint': joint_hp}
cm_model_types = {'transcriptomics': ['SVR_linear', 'SVR_rbf', 'SVR_poly'], 
                 'proteomics': ['SVR_linear', 'SVR_rbf', 'SVR_poly', 'Ridge'], 
                 'joint': ['SVR_linear', 'SVR_rbf', 'SVR_poly', 'Ridge']}

In [58]:
cm_summary = pd.DataFrame(columns = ['modality', 'model_type', 'consensus hyperparameters'])
for modality, df in hp_all.items():
    model_types = cm_model_types[modality]
    df = df[df.model_type.isin(model_types)]
    break_ = False
    for mt in model_types:
        bp = pd.DataFrame(df[df.model_type == mt]['best_params'].apply(lambda x: eval(x)).tolist())
        bp.columns = [x.split('__')[1] for x in bp.columns]

        consensus_bp = {}
        for col in bp.columns:
            if col in ['n_features', 'n_features_rna', 'gamma', 'degree']:
                consensus_bp[col] = bp[col].mode().tolist()[0]
            elif col in ['n_features_protein']:
                consensus_bp[col] = int(bp[col].median())
            elif col in ['C', 'epsilon', 'coef0', 'alpha']:
                consensus_bp[col] = float(bp[col].mean())
            else:
                print(col)
                raise ValueError('Unexpected hyperparameter')
        cm_summary.loc[cm_summary.shape[0], :] = [modality, mt, consensus_bp]

#### Sheets 2-4:

In [59]:
def format_hp_res(df, modality):
    df = df.copy()
    df.drop(columns = ['inner_cv'], inplace = True)
    df['best_params'] = df['best_params'].apply(lambda x: {k.split('__')[1]:v for k,v in eval(x).items()})
    df['modality'] = modality
    
    return df

In [60]:
rna_hp = format_hp_res(rna_hp, modality = 'transcriptomics')
protein_hp = format_hp_res(protein_hp, modality = 'proteomics')
joint_hp = format_hp_res(joint_hp, modality = 'joint')


In [62]:
# Write to Excel with multiple sheets
with pd.ExcelWriter(os.path.join(data_path, 'processed', 'TableWW.xlsx')) as writer:
    hps.to_excel(writer, sheet_name="hyperparameters explored", index=False)
    rna_hp.to_excel(writer, sheet_name="transcriptomics Optuna results", index=False)
    protein_hp.to_excel(writer, sheet_name="proteomics Optuna results", index=False)
    joint_hp.to_excel(writer, sheet_name="joint Optuna results", index=False)
    cm_summary.to_excel(writer, sheet_name="consensus model hyperparameters", index=False)

### Table XX

MWU p-values and Cohen's D comparing each of the performance metrics of the best models (reported in Table WW) across all 10-folds. 

In [52]:
with pd.ExcelWriter(os.path.join(data_path, 'processed', 'TableXX.xlsx')) as writer:
    for modality in ['transcriptomics', 'proteomics', 'joint']:
        df = pd.read_csv(os.path.join(data_path, 'processed', modality + '_comparison_stats.csv'), 
                        index_col = 0)
        df['modality'] = modality
        df.to_excel(writer, sheet_name=modality, index=False)